<a href="https://colab.research.google.com/github/LHallGit/Data-Science-Tools/blob/main/retirement_planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ============================================
# COMPLETE RETIREMENT PLANNER
# One-Cell Solution with Reset
# ============================================

# Install packages (first time only)
import sys
try:
    import ipywidgets
    import plotly
except:
    print("📦 Installing packages... (30 seconds)")
    !pip install -q ipywidgets plotly
    print("✅ Installation complete! Please run this cell again.")
    sys.exit()

# Imports
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dataclasses import dataclass
import json
from datetime import datetime
import base64

# ============================================
# DATA MODELS
# ============================================

@dataclass
class PersonalInfo:
    current_age: int
    retirement_age: int
    life_expectancy: int = 85
    spouse_age: int = 43
    spouse_retirement_age: int = 55

    def years_to_retirement(self):
        return max(0, self.retirement_age - self.current_age)

    def years_in_retirement(self):
        return max(0, self.life_expectancy - self.retirement_age)

@dataclass
class Asset:
    name: str
    current_value: float
    currency: str
    asset_type: str
    expected_return: float = 0.06
    volatility: float = 0.18
    fees: float = 0.0

@dataclass
class IncomeStream:
    name: str
    monthly_amount: float
    currency: str
    start_age: int
    end_age: int
    growth_rate: float = 0.0
    probability: float = 1.0

    def annual_amount(self, age):
        if age < self.start_age or age > self.end_age:
            return 0.0
        years_elapsed = age - self.start_age
        return self.monthly_amount * 12 * ((1 + self.growth_rate) ** years_elapsed)

@dataclass
class Expense:
    category: str
    monthly_amount: float
    currency: str
    start_age: int = 60
    end_age: int = 85
    inflation_rate: float = 0.05

    def annual_amount(self, age):
        if age < self.start_age or age > self.end_age:
            return 0.0
        years_elapsed = age - self.start_age
        return self.monthly_amount * 12 * ((1 + self.inflation_rate) ** years_elapsed)

# ============================================
# MONTE CARLO ENGINE
# ============================================

class MonteCarloSimulation:
    def __init__(self, personal_info, assets, income_streams, expenses, exchange_rates, num_simulations=5000):
        self.personal_info = personal_info
        self.assets = assets
        self.income_streams = income_streams
        self.expenses = expenses
        self.exchange_rates = exchange_rates
        self.num_simulations = num_simulations
        self.years_in_retirement = personal_info.years_in_retirement()
        np.random.seed(42)

    def run(self):
        initial_value = sum(
            asset.current_value * self.exchange_rates.get(asset.currency, 1.0)
            for asset in self.assets
        )

        portfolio_paths = np.zeros((self.num_simulations, self.years_in_retirement + 1))

        for sim in range(self.num_simulations):
            portfolio_paths[sim] = self._run_single_simulation(initial_value)

            if (sim + 1) % 500 == 0:
                print(f"  ✓ {sim + 1:,}/{self.num_simulations:,} simulations...")

        final_portfolios = portfolio_paths[:, -1]
        success_count = np.sum(final_portfolios >= 1_000_000)

        return {
            'portfolio_paths': portfolio_paths,
            'success_probability': success_count / self.num_simulations,
            'median_final': np.median(final_portfolios),
            'percentile_10': np.percentile(final_portfolios, 10),
            'percentile_90': np.percentile(final_portfolios, 90),
            'initial_value': initial_value
        }

    def _run_single_simulation(self, initial_value):
        portfolio_value = initial_value
        path = np.zeros(self.years_in_retirement + 1)
        path[0] = initial_value

        for year in range(1, self.years_in_retirement + 1):
            age = self.personal_info.retirement_age + year - 1
            annual_return = np.random.normal(0.06, 0.18)
            portfolio_value *= (1 + annual_return)

            expenses = sum(
                exp.annual_amount(age) * self.exchange_rates.get(exp.currency, 1.0)
                for exp in self.expenses
            )

            income = sum(
                inc.annual_amount(age) * self.exchange_rates.get(inc.currency, 1.0)
                if np.random.random() < inc.probability else 0
                for inc in self.income_streams
            )

            portfolio_value += income - expenses
            portfolio_value = max(0, portfolio_value)
            path[year] = portfolio_value

        return path

# ============================================
# COUNTRY DATA
# ============================================

COUNTRY_EXPENSES = {
    'South Africa': {
        'Groceries': 12000, 'Healthcare': 1800, 'Property Levies': 1600,
        'Property Rates': 200, 'Utilities': 3000, 'Vehicle Payment': 5000,
        'Vehicle Insurance': 1000, 'Fuel': 1500, 'Entertainment': 1500,
        'Hobbies': 800, 'Travel': 6667,
        'Currency': 'ZAR', 'Inflation': 0.05, 'Tax_Rate': 0.31
    },
    'Fiji': {
        'Groceries': 2500, 'Healthcare': 200, 'Housing': 3500,
        'Utilities': 700, 'Transportation': 800, 'Entertainment': 500,
        'Travel': 1000,
        'Currency': 'FJD', 'Inflation': 0.05, 'Tax_Rate': 0.0
    },
    'Panama': {
        'Groceries': 400, 'Healthcare': 250, 'Housing': 1200,
        'Utilities': 80, 'Transportation': 300, 'Entertainment': 200,
        'Travel': 300,
        'Currency': 'USD', 'Inflation': 0.018, 'Tax_Rate': 0.0
    }
}

# ============================================
# CREATE ALL WIDGETS
# ============================================

# Output areas
main_output = widgets.Output()
comparison_output = widgets.Output()

# Personal Info
age_slider = widgets.IntSlider(value=55, min=30, max=80, description='Current Age:', style={'description_width': '120px'})
retirement_slider = widgets.IntSlider(value=60, min=55, max=70, description='Retire Age:', style={'description_width': '120px'})
life_expectancy_slider = widgets.IntSlider(value=85, min=70, max=100, description='Life Expect:', style={'description_width': '120px'})

# Country
country_dropdown = widgets.Dropdown(
    options=['South Africa', 'Fiji', 'Panama'],
    value='South Africa',
    description='Country:',
    style={'description_width': '120px'}
)

# Assets
nyse_input = widgets.FloatText(value=1265, description='NYSE (USD):', style={'description_width': '120px'})
jse_input = widgets.FloatText(value=25000, description='JSE (ZAR):', style={'description_width': '120px'})
tfsa_input = widgets.FloatText(value=19800, description='TFSA (ZAR):', style={'description_width': '120px'})
fnpf_input = widgets.FloatText(value=291916, description='FNPF (FJD):', style={'description_width': '120px'})
mondi_pension_input = widgets.FloatText(value=8156696, description='Mondi Pension:', style={'description_width': '120px'})
mondi_provident_input = widgets.FloatText(value=916261, description='Mondi Prov:', style={'description_width': '120px'})
house_input = widgets.FloatText(value=1250000, description='SA House:', style={'description_width': '120px'})

# Income
salary_input = widgets.FloatText(value=13000, description='Salary (FJD):', style={'description_width': '120px'})
celeste_salary_input = widgets.FloatText(value=10000, description='Celeste (ZAR):', style={'description_width': '120px'})
rental_input = widgets.FloatText(value=5400, description='Rental (ZAR):', style={'description_width': '120px'})
part_time_input = widgets.FloatText(value=15000, description='Part-Time:', style={'description_width': '120px'})

# Expense info display
expense_info = widgets.HTML()

# All buttons
run_button = widgets.Button(description='🎲 RUN SIMULATION', button_style='success', layout=widgets.Layout(width='280px', height='50px'))
compare_button = widgets.Button(description='🌍 COMPARE COUNTRIES', button_style='warning', layout=widgets.Layout(width='280px', height='50px'))
expense_button = widgets.Button(description='📊 EXPENSE BREAKDOWN', button_style='info', layout=widgets.Layout(width='280px', height='40px'))
export_button = widgets.Button(description='📄 EXPORT REPORT', button_style='primary', layout=widgets.Layout(width='280px', height='40px'))
save_button = widgets.Button(description='💾 SAVE SCENARIO', button_style='primary', layout=widgets.Layout(width='280px', height='40px'))
reset_button = widgets.Button(description='🔄 RESET ALL', button_style='danger', layout=widgets.Layout(width='280px', height='40px'))

load_button = widgets.FileUpload(accept='.json', multiple=False, description='📂 LOAD')

# Results storage
results_data = {'South Africa': None, 'Fiji': None, 'Panama': None}

# ============================================
# HELPER FUNCTIONS
# ============================================

def get_healthcare_multiplier(age):
    if age <= 65: return 1.0
    elif age <= 70: return 1.3
    elif age <= 75: return 1.6
    elif age <= 80: return 2.0
    else: return 2.5

def get_expenses_for_country(country, retirement_age):
    exp_data = COUNTRY_EXPENSES[country]
    currency = exp_data['Currency']
    inflation = exp_data['Inflation']

    expenses = []
    for category, amount in exp_data.items():
        if category not in ['Currency', 'Inflation', 'Tax_Rate']:
            end_age = min(retirement_age + 5, 85) if category == 'Vehicle Payment' else 85
            expenses.append(Expense(category, amount, currency, retirement_age, end_age, inflation))

    return expenses

def update_expense_info(change):
    country = country_dropdown.value
    expenses = COUNTRY_EXPENSES[country]
    currency = expenses['Currency']
    inflation = expenses['Inflation'] * 100
    tax = expenses['Tax_Rate'] * 100
    total = sum(v for k, v in expenses.items() if k not in ['Currency', 'Inflation', 'Tax_Rate'])

    html = f"""
    <div style="background-color: #e8f4f8; padding: 15px; border-radius: 5px; border-left: 4px solid #2196F3;">
        <h4 style="margin-top: 0; color: #1976D2;">📍 {country}</h4>
        <table style="width: 100%; font-size: 14px;">
            <tr><td><b>Currency:</b></td><td>{currency}</td></tr>
            <tr><td><b>Inflation:</b></td><td>{inflation:.1f}%</td></tr>
            <tr><td><b>Tax Rate:</b></td><td>{tax:.0f}%</td></tr>
            <tr><td><b>Monthly:</b></td><td><b>{total:,.0f} {currency}</b></td></tr>
            <tr><td><b>Annual:</b></td><td><b>{total * 12:,.0f} {currency}</b></td></tr>
        </table>
    </div>
    """
    expense_info.value = html

def run_single_country(country, show_output=True):
    EXCHANGE_RATES = {"USD": 17.50, "FJD": 7.80, "ZAR": 1.0}

    personal_info = PersonalInfo(
        current_age=age_slider.value,
        retirement_age=retirement_slider.value,
        life_expectancy=life_expectancy_slider.value
    )

    assets = [
        Asset("NYSE", nyse_input.value, "USD", "equity", 0.08, 0.18),
        Asset("JSE", jse_input.value, "ZAR", "equity", 0.08, 0.18),
        Asset("TFSA", tfsa_input.value, "ZAR", "equity", 0.08, 0.18),
        Asset("FNPF", fnpf_input.value, "FJD", "pension", 0.075, 0.10),
        Asset("Mondi Pension", mondi_pension_input.value, "ZAR", "pension", 0.06, 0.15, 0.01),
        Asset("Mondi Provident", mondi_provident_input.value, "ZAR", "pension", 0.06, 0.15),
        Asset("SA House", house_input.value, "ZAR", "property", 0.03, 0.05),
    ]

    income_streams = [
        IncomeStream("Salary", salary_input.value, "FJD", age_slider.value, retirement_slider.value, 0.01),
        IncomeStream("Celeste", celeste_salary_input.value, "ZAR", 43, 55, 0.03),
        IncomeStream("Rental", rental_input.value, "ZAR", age_slider.value, 85, 0.03),
        IncomeStream("Part-Time", part_time_input.value, "ZAR", retirement_slider.value, 68, 0.0, 0.65),
    ]

    expenses = get_expenses_for_country(country, retirement_slider.value)

    if show_output:
        print(f"\n🌍 Simulating {country}...")

    simulation = MonteCarloSimulation(personal_info, assets, income_streams, expenses, EXCHANGE_RATES, 5000)
    results = simulation.run()
    results_data[country] = results

    return results

# ============================================
# BUTTON HANDLERS
# ============================================

def run_simulation(b):
    with main_output:
        clear_output()
        country = country_dropdown.value

        print("="*60)
        print(f"   🎲 SIMULATING: {country.upper()}")
        print("="*60)

        results = run_single_country(country)

        print("\n" + "="*60)
        print("📊 RESULTS")
        print("="*60)
        print(f"✅ Success: {results['success_probability']:.1%}")
        print(f"💰 Median: R {results['median_final']:,.0f}")
        print(f"📉 10th %: R {results['percentile_10']:,.0f}")
        print(f"📈 90th %: R {results['percentile_90']:,.0f}")
        print("="*60)

        if results['success_probability'] >= 0.85:
            print("\n✅ EXCELLENT: Very secure!")
        elif results['success_probability'] >= 0.70:
            print("\n⚠️ GOOD: Moderate risk.")
        else:
            print("\n🚨 HIGH RISK: Adjustments needed!")

        # Chart
        paths = results['portfolio_paths']
        years = paths.shape[1]
        median = np.percentile(paths, 50, axis=0)
        p10 = np.percentile(paths, 10, axis=0)
        p90 = np.percentile(paths, 90, axis=0)

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=list(range(years)), y=p90, mode='lines', name='90th', line=dict(width=0), showlegend=False))
        fig.add_trace(go.Scatter(x=list(range(years)), y=p10, mode='lines', name='10th-90th', fill='tonexty', fillcolor='rgba(68,68,68,0.2)', line=dict(width=0)))
        fig.add_trace(go.Scatter(x=list(range(years)), y=median, mode='lines', name='Median', line=dict(color='blue', width=3)))
        fig.add_trace(go.Scatter(x=list(range(years)), y=[1_000_000]*years, mode='lines', name='Threshold', line=dict(color='red', width=2, dash='dash')))

        fig.update_layout(title=f"{country} - Portfolio Projection", xaxis_title="Years", yaxis_title="ZAR", height=500)
        fig.show()

def compare_countries(b):
    with comparison_output:
        clear_output()
        print("="*60)
        print("   🌍 COMPARING ALL COUNTRIES")
        print("="*60)

        countries = ['South Africa', 'Fiji', 'Panama']
        all_results = {}

        for country in countries:
            print(f"\n📍 {country}:")
            results = run_single_country(country, show_output=False)
            all_results[country] = results
            print(f"   ✅ Success: {results['success_probability']:.1%}")
            print(f"   💰 Median: R {results['median_final']:,.0f}")

        # Comparison table
        print("\n" + "="*60)
        print("📊 COMPARISON")
        print("="*60)

        df = pd.DataFrame({
            'Metric': ['Success %', 'Median Final (ZAR)', '10th % (ZAR)', '90th % (ZAR)'],
            'South Africa': [
                f"{all_results['South Africa']['success_probability']:.1%}",
                f"R {all_results['South Africa']['median_final']:,.0f}",
                f"R {all_results['South Africa']['percentile_10']:,.0f}",
                f"R {all_results['South Africa']['percentile_90']:,.0f}"
            ],
            'Fiji': [
                f"{all_results['Fiji']['success_probability']:.1%}",
                f"R {all_results['Fiji']['median_final']:,.0f}",
                f"R {all_results['Fiji']['percentile_10']:,.0f}",
                f"R {all_results['Fiji']['percentile_90']:,.0f}"
            ],
            'Panama': [
                f"{all_results['Panama']['success_probability']:.1%}",
                f"R {all_results['Panama']['median_final']:,.0f}",
                f"R {all_results['Panama']['percentile_10']:,.0f}",
                f"R {all_results['Panama']['percentile_90']:,.0f}"
            ]
        })

        print("\n" + df.to_string(index=False))

        # Best option
        best = max(all_results.keys(), key=lambda k: all_results[k]['success_probability'])
        print(f"\n🏆 BEST OPTION: {best} ({all_results[best]['success_probability']:.1%})")

        # Chart
        fig = make_subplots(rows=1, cols=3, subplot_titles=('🇿🇦 SA', '🇫🇯 Fiji', '🇵🇦 Panama'))

        for idx, country in enumerate(countries, 1):
            median = np.percentile(all_results[country]['portfolio_paths'], 50, axis=0)
            years = len(median)
            fig.add_trace(go.Scatter(x=list(range(years)), y=median, name=country, line=dict(width=3)), row=1, col=idx)
            fig.add_trace(go.Scatter(x=list(range(years)), y=[1_000_000]*years, name='Threshold', line=dict(color='red', dash='dash'), showlegend=False), row=1, col=idx)

        fig.update_layout(title="Country Comparison", height=400, showlegend=False)
        fig.show()

def show_expense_breakdown(b):
    with main_output:
        clear_output()
        country = country_dropdown.value

        print("="*70)
        print(f"📊 EXPENSE BREAKDOWN - {country.upper()}")
        print("="*70)

        EXCHANGE_RATES = {"USD": 17.50, "FJD": 7.80, "ZAR": 1.0}
        exp_data = COUNTRY_EXPENSES[country]
        currency = exp_data['Currency']
        inflation = exp_data['Inflation']
        conversion = EXCHANGE_RATES[currency]

        ages = [60, 65, 70, 75, 80, 85]
        breakdown = []

        for age in ages:
            years = age - 60
            total_zar = 0
            row = {'Age': age}

            for cat, amt in exp_data.items():
                if cat not in ['Currency', 'Inflation', 'Tax_Rate']:
                    inflated = amt * ((1 + inflation) ** years)
                    if cat == 'Healthcare':
                        inflated *= get_healthcare_multiplier(age)
                    row[cat] = inflated
                    total_zar += inflated * conversion

            row['Total (ZAR)'] = total_zar
            breakdown.append(row)

        df = pd.DataFrame(breakdown)
        print(f"\nAmounts in {currency}:\n")
        print(df.to_string(index=False))

        print("\n💊 Healthcare escalation: 60-65(1x), 66-70(1.3x), 71-75(1.6x), 76-80(2x), 81-85(2.5x)")

def export_report(b):
    with main_output:
        clear_output()
        country = country_dropdown.value

        if results_data[country] is None:
            print("⚠️ Run simulation first!")
            return

        results = results_data[country]

        report = f"""
{'='*70}
              RETIREMENT PLANNING REPORT
{'='*70}

Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}
Country: {country}

PERSONAL INFO:
  Current Age: {age_slider.value}
  Retirement Age: {retirement_slider.value}
  Life Expectancy: {life_expectancy_slider.value}

ASSETS (Total: R {results['initial_value']:,.0f}):
  NYSE: ${nyse_input.value:,.0f}
  JSE: R {jse_input.value:,.0f}
  TFSA: R {tfsa_input.value:,.0f}
  FNPF: FJ$ {fnpf_input.value:,.0f}
  Mondi Pension: R {mondi_pension_input.value:,.0f}
  Mondi Provident: R {mondi_provident_input.value:,.0f}
  SA House: R {house_input.value:,.0f}

RESULTS:
  Success Probability: {results['success_probability']:.1%}
  Median Final: R {results['median_final']:,.0f}
  10th Percentile: R {results['percentile_10']:,.0f}
  90th Percentile: R {results['percentile_90']:,.0f}

{'='*70}
"""

        print(report)

        # Download link
        b64 = base64.b64encode(report.encode()).decode()
        filename = f'report_{country.replace(" ", "_")}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.txt'

        html = f'<a download="{filename}" href="data:text/plain;base64,{b64}" style="background-color: #28a745; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; display: inline-block;">📥 Download Report</a>'
        display(HTML(html))

def save_scenario(b):
    with main_output:
        clear_output()

        scenario = {
            'name': f'Scenario_{datetime.now().strftime("%Y%m%d_%H%M%S")}',
            'date': datetime.now().isoformat(),
            'personal': {'age': age_slider.value, 'retire': retirement_slider.value, 'life': life_expectancy_slider.value},
            'assets': {
                'nyse': nyse_input.value, 'jse': jse_input.value, 'tfsa': tfsa_input.value,
                'fnpf': fnpf_input.value, 'mondi_pension': mondi_pension_input.value,
                'mondi_provident': mondi_provident_input.value, 'house': house_input.value
            },
            'income': {
                'salary': salary_input.value, 'celeste': celeste_salary_input.value,
                'rental': rental_input.value, 'part_time': part_time_input.value
            },
            'country': country_dropdown.value
        }

        json_str = json.dumps(scenario, indent=2)
        b64 = base64.b64encode(json_str.encode()).decode()
        filename = f'scenario_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'

        print("✅ Scenario saved!\n")
        html = f'<a download="{filename}" href="data:application/json;base64,{b64}" style="background-color: #007bff; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; display: inline-block;">📥 Download Scenario</a>'
        display(HTML(html))

def load_scenario(change):
    if load_button.value:
        try:
            uploaded = list(load_button.value.values())[0]
            json_text = uploaded['content'].decode('utf-8')
            data = json.loads(json_text)

            age_slider.value = data['personal']['age']
            retirement_slider.value = data['personal']['retire']
            life_expectancy_slider.value = data['personal']['life']

            nyse_input.value = data['assets']['nyse']
            jse_input.value = data['assets']['jse']
            tfsa_input.value = data['assets']['tfsa']
            fnpf_input.value = data['assets']['fnpf']
            mondi_pension_input.value = data['assets']['mondi_pension']
            mondi_provident_input.value = data['assets']['mondi_provident']
            house_input.value = data['assets']['house']

            salary_input.value = data['income']['salary']
            celeste_salary_input.value = data['income']['celeste']
            rental_input.value = data['income']['rental']
            part_time_input.value = data['income']['part_time']

            country_dropdown.value = data['country']

            with main_output:
                clear_output()
                print("✅ Scenario loaded successfully!")
                print(f"Name: {data['name']}")
                print(f"Created: {data['date']}")
                print(f"Country: {data['country']}")

        except Exception as e:
            with main_output:
                clear_output()
                print(f"❌ Error loading: {str(e)}")

def reset_all(b):
    with main_output:
        clear_output()
        print("🔄 Resetting all inputs to default values...")

    with comparison_output:
        clear_output()

    # Reset all values
    age_slider.value = 55
    retirement_slider.value = 60
    life_expectancy_slider.value = 85

    nyse_input.value = 1265
    jse_input.value = 25000
    tfsa_input.value = 19800
    fnpf_input.value = 291916
    mondi_pension_input.value = 8156696
    mondi_provident_input.value = 916261
    house_input.value = 1250000

    salary_input.value = 13000
    celeste_salary_input.value = 10000
    rental_input.value = 5400
    part_time_input.value = 15000

    country_dropdown.value = 'South Africa'

    # Clear results
    results_data['South Africa'] = None
    results_data['Fiji'] = None
    results_data['Panama'] = None

    with main_output:
        print("✅ Reset complete! Ready for new simulation.")

# Connect buttons
run_button.on_click(run_simulation)
compare_button.on_click(compare_countries)
expense_button.on_click(show_expense_breakdown)
export_button.on_click(export_report)
save_button.on_click(save_scenario)
reset_button.on_click(reset_all)
load_button.observe(load_scenario, names='value')
country_dropdown.observe(update_expense_info, 'value')
update_expense_info(None)

# ============================================
# GUI LAYOUT
# ============================================

tab1 = widgets.VBox([
    widgets.HTML("<h3 style='color: #1976D2;'>👤 Personal</h3>"),
    age_slider, retirement_slider, life_expectancy_slider
], layout=widgets.Layout(padding='10px'))

tab2 = widgets.VBox([
    widgets.HTML("<h3 style='color: #1976D2;'>💼 Assets</h3>"),
    nyse_input, jse_input, tfsa_input, fnpf_input,
    mondi_pension_input, mondi_provident_input, house_input
], layout=widgets.Layout(padding='10px'))

tab3 = widgets.VBox([
    widgets.HTML("<h3 style='color: #1976D2;'>💵 Income</h3>"),
    salary_input, celeste_salary_input, rental_input, part_time_input
], layout=widgets.Layout(padding='10px'))

tab4 = widgets.VBox([
    widgets.HTML("<h3 style='color: #1976D2;'>🌍 Country</h3>"),
    country_dropdown, expense_info
], layout=widgets.Layout(padding='10px'))

tabs = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tabs.set_title(0, '👤 Personal')
tabs.set_title(1, '💼 Assets')
tabs.set_title(2, '💵 Income')
tabs.set_title(3, '🌍 Country')

# Button rows
main_buttons = widgets.VBox([
    widgets.HBox([run_button, widgets.HTML("<div style='width: 20px;'></div>"), compare_button], layout=widgets.Layout(justify_content='center')),
    widgets.HTML("<div style='height: 10px;'></div>"),
    widgets.HBox([expense_button, widgets.HTML("<div style='width: 20px;'></div>"), export_button], layout=widgets.Layout(justify_content='center')),
    widgets.HTML("<div style='height: 10px;'></div>"),
    widgets.HBox([save_button, widgets.HTML("<div style='width: 20px;'></div>"), load_button, widgets.HTML("<div style='width: 20px;'></div>"), reset_button], layout=widgets.Layout(justify_content='center'))
], layout=widgets.Layout(padding='20px'))

# ============================================
# DISPLAY
# ============================================

print("="*70)
print("      💰 MULTI-COUNTRY RETIREMENT PLANNER 💰")
print("="*70)
print("\n🎯 FEATURES:")
print("  • Monte Carlo simulation (5,000 runs)")
print("  • Multi-country comparison (SA, Fiji, Panama)")
print("  • Healthcare cost escalation modeling")
print("  • Tax analysis for each country")
print("  • Save/load scenarios")
print("  • Export detailed reports")
print("\n📋 QUICK START:")
print("  1. Adjust your data in the tabs")
print("  2. Click '🎲 RUN SIMULATION' for single country")
print("  3. Click '🌍 COMPARE COUNTRIES' for side-by-side")
print("  4. Click '🔄 RESET ALL' to start fresh")
print("\n" + "="*70 + "\n")

display(tabs)
display(main_buttons)
display(widgets.HTML("<hr><h3 style='text-align: center;'>📊 RESULTS</h3>"))
display(main_output)
display(widgets.HTML("<hr><h3 style='text-align: center;'>🌍 COUNTRY COMPARISON</h3>"))
display(comparison_output)

print("\n✅ Ready! Use the buttons above to run simulations.")

      💰 MULTI-COUNTRY RETIREMENT PLANNER 💰

🎯 FEATURES:
  • Monte Carlo simulation (5,000 runs)
  • Multi-country comparison (SA, Fiji, Panama)
  • Healthcare cost escalation modeling
  • Tax analysis for each country
  • Save/load scenarios
  • Export detailed reports

📋 QUICK START:
  1. Adjust your data in the tabs
  2. Click '🎲 RUN SIMULATION' for single country
  3. Click '🌍 COMPARE COUNTRIES' for side-by-side
  4. Click '🔄 RESET ALL' to start fresh




HTML(value="<hr><h3 style='text-align: center;'>📊 RESULTS</h3>")

Output()

HTML(value="<hr><h3 style='text-align: center;'>🌍 COUNTRY COMPARISON</h3>")

Output()


✅ Ready! Use the buttons above to run simulations.
